In [1]:
from pathlib import Path
this_path = Path().resolve()

import torch
import numpy as np
from torch import nn, optim
from models.medicalnet.model import generate_model
from models.medicalnet.setting import get_def_args


/home/alejandrocu/Documents/parkinson_classification/models/medicalnet


In [2]:
sets = get_def_args()   
model, parameters = generate_model(sets) 

print (model)

loading pretrained model pretrain/resnet_10_23dataset.pth
ResNet(
  (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=Fals

In [3]:
model.conv_seg = nn.Sequential(
                                nn.AdaptiveMaxPool3d(output_size=(1, 1, 1)),
                                nn.Flatten(start_dim=1),
                                nn.Dropout(0.1),
                                # the last Conv3d layer has out_channels = 512
                                nn.Linear(512, 2)
                                )

In [6]:
input = torch.randn(3, 1, 200, 200, 200).cuda()
output = model(input)
print(output.shape)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)

In [30]:
m = nn.Flatten()
print(m(input).shape)

torch.Size([3, 8000000])


In [12]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    # print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [13]:
count_parameters(model)

Total Trainable Params: 14341666


14341666

In [19]:
import monai
net = monai.networks.nets.EfficientNetBN('efficientnet-b0', 
                                                    spatial_dims=3, 
                                                    in_channels=3, 
                                                    num_classes=2)
count_parameters(net)
print([x for x in net.named_modules()])                      

Total Trainable Params: 4692670
[('', EfficientNetBN(
  (_conv_stem): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), bias=False)
  (_conv_stem_padding): ConstantPad3d(padding=(0, 1, 0, 1, 0, 1), value=0.0)
  (_bn0): BatchNorm3d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (_blocks): Sequential(
    (0): Sequential(
      (0): MBConvBlock(
        (_expand_conv): Identity()
        (_expand_conv_padding): Identity()
        (_bn0): Identity()
        (_depthwise_conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), groups=32, bias=False)
        (_depthwise_conv_padding): ConstantPad3d(padding=(1, 1, 1, 1, 1, 1), value=0.0)
        (_bn1): BatchNorm3d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (_se_adaptpool): AdaptiveAvgPool3d(output_size=1)
        (_se_reduce): Conv3d(32, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1))
        (_se_reduce_padding): Identity()
        (_se_expand): Conv3d(8, 32, kernel_size=(1, 1

In [20]:
import torchio as tio
aug = tio.Compose([
                tio.RandomAffine(translation=10),
                tio.RandomElasticDeformation(p=0.1, num_control_points=7, max_displacement=10),
                tio.RandomGamma(p=0.5),
                tio.RandomNoise(p=0.5, mean=0.5, std=0.05), # p=0.5
                tio.RandomMotion(p=0.1), #, degrees=20, translation=20),
                tio.RandomBiasField(p=0.25),
                ])

In [25]:
aug.transforms[0].args_names

[]